# Gateway Lesson


This is a PGC Data Workshop Gateway Lesson, which is your introduction to a foundational concept _coregistration_ and key skills to model a glacier elevation change. This lesson prepares you for topics such as analysis of DEM accuracy, differnecing, and DEM coregistration. So with that said:

## Welcome to the Coregistration Exercise!

<br>

Lesson Developer: 
  * Ali Hossaini (hossa084@umn.edu)


# DEM: Gridded Representations of Elevation

Digital Elevation Models (DEMs) serve as numerical, gridded representations of elevation, deriving from diverse instruments (e.g., optical sensors, radar, lidar). They are acquired under varying conditions, including ground, airborne, and satellite platforms, and are processed using different techniques like photogrammetry and remote sensing.

!["DEM"](An-example-of-a-Digital-Elevation-Model.png#center)
> <span style="color:gray">
image source: Barcellos, Bruno & Giraldi, Gilson & Silva, Rodrigo & Apolinario Jr, Antonio L. & Rodrigues, Paulo. (2007). Surface Flow Animation in Digital Terrain Models. </span>


While specific complexities may be tied to certain instruments and methods, common characteristics of DEMs include:

- **Ground Sampling Distance (GSD):** The pixel size, or GSD, doesn't always reflect the actual spatial resolution of the observations.

- **Georeferencing Challenges:** Georeferencing is prone to shifts, tilts, or deformations due to inherent instrument errors, noise, or associated processing schemes.

- **Outlier Abundance:** DEMs often contain numerous outliers that are challenging to filter, stemming from diverse sources like photogrammetric errors and cloud interference.

These factors contribute to the complexity of assessing the <b>accuracy</b> and <b>precision</b> of DEMs, a crucial step for any subsequent analysis.

!["DEM"](800px-Grancanaria-meshlab-vs-meshmixer.png#center)
> <span style="color:gray">
    image source: edutechwiki.unige.ch </span>



## All DEMs have some horizontal and vertical geolocation error. 

The **spatial structure of DEMs** introduces complexity to the concepts of <b>accuracy</b> and <b>precision</b>. For example:

Spatially structured systematic errors are often associated with the gridded nature of DEMs, leading to affine biases. Additionally, specific biases can exist at the pixel scale. Concerning random errors, there is a variability in error magnitude or heteroscedasticity across the DEM, and spatially structured error patterns are connected to spatial correlations.

**DEM Accuracy (Systematic Error):** Describes the proximity of a Digital Elevation Model (DEM) to the accurate location of measured elevations on the Earth's surface.

**DEM Precision (Random Error):** Represents the typical spread of measurement errors in a DEM, irrespective of any potential bias from accurate positioning.

!["DEM"](Pattern_of_Errors.png#center)
>[ <span style="color:gray">
    image source: Hugonnet et al. (2022) </span>](https://ieeexplore.ieee.org/document/9815885)

**Optimizing DEM Accuracy in relation to other georeferenced data:**

Shifts to the true positioning resulting from poor accuracy are common in elevation datasets and can be corrected effectively by performign a **DEM co-registration** with precise and accurate refrence elevation data that undergoes quality control. Quality-controlled DEMs, aligned to high-accuracy data, are also available, such as TanDEM-X global DEM (refer to [Rizzoli et al. (2017)](https://www.sciencedirect.com/science/article/pii/S092427161730093X?via%3Dihub)).

These biases can be addressed using the techniques outlined in **Coregistration**.


!["DEM"](Co1.jpg#center)
!["DEM"](Co2.jpg#center)
> [<span style="color:gray">
    image source: Nuth and Kääb (2011) </span> ](https://tc.copernicus.org/articles/5/271/2011/tc-5-271-2011.pdf)

## Coregistration

Coregistration aligns multiple datasets to a common reference, minimizing geometric differences. This involves adjusting the spatial parameters of one dataset to match those of another which: 

- may include transformations such as translation, rotation, scaling, and possibly more complex adjustments to account for distortions in the data.

- is often necessary when merging elevation data from various sources, such as satellite imagery, aerial photographs, LiDAR, or other remote sensing technologies. 

- each dataset may have been acquired at different times, from different platforms, or using different sensors, resulting in variations in spatial orientation, scale, and geometry.


## DEM Coregistration
Coregistration between DEMs correspond to aligning the digital elevation models in three dimensions.

Transformations that can be described by a 3-dimensional affine function are included in coregistration methods which include:

- vertical and horizontal translations

- rotations

- scalings.


## Today's Focus: Greenland's Glacial Topography

Materials Required:

Two Digital Elevation Models (DEMs) of Greenland (can be obtained from different sources, e.g., NASA DEMs, ArcticDEM, etc.)

Jupyter Notebook with Python and relevant libraries (e.g., rasterio, geopandas, matplotlib)



## Install/Import Libraries:

Import necessary Python libraries for geospatial analysis by running the following cells (Shift+Enter).



In [3]:
import rasterio
from rasterio.enums import Resampling
import geopandas as gpd
import folium
import plotly.express as px

# Load DEMs:

Load the two DEMs of Greenland using rasterio.


In [ ]:
dem_path1 = 'DEM/dem1.tif'
dem_path2 = 'DEM/reference_dem.tif'
dem1 = rasterio.open(dem_path1)
dem2 = rasterio.open(dem_path2)


# Explore Initial Alignment:

Display the two DEMs to observe the misalignment.



In [ ]:
def display_dem_on_map(dem, map_center):
    m = folium.Map(location=map_center, zoom_start=6, control_scale=True)
    folium.raster_layers.ImageOverlay(
        image=dem.read(1),
        bounds=[[dem.bounds.bottom, dem.bounds.left], [dem.bounds.top, dem.bounds.right]],
        colormap='terrain',
    ).add_to(m)
    return m

center_coord = [(dem1.bounds.bottom + dem1.bounds.top) / 2, (dem1.bounds.left + dem1.bounds.right) / 2]
map_dem1 = display_dem_on_map(dem1, center_coord)
map_dem2 = display_dem_on_map(dem2, center_coord)


# Perform Coregistration:

Coregister one DEM to the other using resampling and transformation.



In [ ]:
dem2_coregistered, transform = rasterio.warp.reproject(
    source=rasterio.band(dem2, 1),
    destination=rasterio.band(dem1, 1),
    src_transform=dem2.transform,
    src_crs=dem2.crs,
    dst_transform=dem1.transform,
    dst_crs=dem1.crs,
    resampling=Resampling.bilinear
)


# Interactive  Coregistered DEM:

Display the coregistered DEM.



In [ ]:
def display_coregistered_dem_on_map(coregistered_dem, transform, map_center):
    m = folium.Map(location=map_center, zoom_start=6, control_scale=True)
    bounds = rasterio.transform.array_bounds(coregistered_dem.shape[0], coregistered_dem.shape[1], transform)
    folium.raster_layers.ImageOverlay(
        image=coregistered_dem,
        bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
        colormap='terrain',
    ).add_to(m)
    return m

map_coregistered_dem = display_coregistered_dem_on_map(dem2_coregistered, transform, center_coord)


# Visualize with Plotly:

Display the coregistered DEM.

In [ ]:
def plot_dem_histogram(dem):
    fig = px.histogram(dem.read(1).flatten(), nbins=50, title='Elevation Histogram')
    fig.update_layout(xaxis_title='Elevation', yaxis_title='Frequency')
    return fig

histogram_plot = plot_dem_histogram(dem2_coregistered)


# Save Coregistered DEM:

Save the coregistered DEM for future use.

In [ ]:
output_path = 'path/to/coregistered_dem.tif'
with rasterio.open(output_path, 'w', driver='GTiff', height=dem1.shape[0], width=dem1.shape[1],
                   count=1, dtype='float32', crs=dem1.crs, transform=transform) as dst:
    dst.write(dem2_coregistered, 1)


# Advanced Method:

Using **geowombat** package to coregister DEMs:

In [4]:
#import library
import geowombat as gw
import matplotlib.pyplot as plt
fig, ax = plt.subplots(dpi=200)

#load data
Refernce_DEM = 'DEM/SETSM_s2s041_WV02_20150402_103001003E9F4300_103001004013BE00_2m_seg1_dem_10m_shade.tif'
Target_DEM = 'DEM/TDX_Greenland_Iceland_Mosaic_3413_90m_Clip.tif'

#coregister and visualize
with gw.config.update(ref_image=Refernce_DEM):
    with gw.open(Target_DEM, resampling="bilinear", nodata=-9999) as src:
        print(src)
        ax.imshow(src.data[0])
        

ModuleNotFoundError: No module named 'geowombat'